# Homework 2 - Top Universities world ranking

# Task 1

### Importing libs

In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

import requests
from bs4 import BeautifulSoup
import folium
import json

# Extracting data from the website

In [2]:
addresses = r'./panama_csv/Addresses.csv'
entities = r'./panama_csv/Entities.csv'
coordinates = r'./panama_csv/coordinates.csv'

addresses = pd.read_csv(addresses,index_col='address', header=0, low_memory=False)
entities = pd.read_csv(entities,index_col='name', header=0, low_memory=False)
coordinates = pd.read_csv(coordinates, index_col='Country', header=0, low_memory=False)

In [3]:
entities=entities.rename(columns = {'countries':'Country'})

In [4]:
coord_entities = entities.join(coordinates, on='Country')

In [5]:
vals = coord_entities.groupby('Country').count().sort_values('original_name', ascending=False).head(50)

In [6]:
lat = coordinates.loc['Switzerland'].at['Latitude (average)']
long = coordinates.loc['Switzerland'].at['Longitude (average)']

In [7]:
vals.index.values
m = folium.Map()

In [8]:
threshold = 5000
for country in vals.index.values:
    try:
        traffic = (vals.loc[country].at['Numeric code'])
        radius = int(traffic / 3000)
        color = 'blue'
        fill = 'lightblue'
        if (traffic > threshold):
            color = 'red'
            fill = 'lightred'
        lat = (coordinates.loc[country].at['Latitude (average)'])
        long = (coordinates.loc[country].at['Longitude (average)'])
        folium.CircleMarker(location=[lat, long], radius=radius, popup=country, icon=folium.Icon(color='crimson')).add_to(m)
    except:
        print(country + " not in list")


Russia not in list
Taiwan not in list
British Virgin Islands not in list
Venezuela not in list


In [9]:
m